<a href="https://colab.research.google.com/github/Alg0r1thmic/IA-Topics/blob/master/inceptionv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:


from keras.models import Model
from keras.layers import Activation, Input, concatenate, Dropout, Dense, Flatten
from keras.layers import MaxPooling2D, Conv2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

import os
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#import itertools
#import shutil
%matplotlib inline


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
PATH="/content/drive/My Drive/topicos en ia" #la carpeta esta compartida
DATASET_DIR=PATH+"/covid-19-x-ray-10000-images/dataset"
IMG_W = 299
IMG_H = 299
CHANNELS = 3

INPUT_SHAPE = (IMG_W, IMG_H, CHANNELS)
NB_CLASSES = 2
EPOCHS = 41
BATCH_SIZE = 6

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        rescale = 1./255,
        validation_split = 0.3) 
train_generator = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_H, IMG_W),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training')
validation_generator = datagen.flow_from_directory(
    DATASET_DIR, 
    target_size=(IMG_H, IMG_W),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle= False,
    subset='validation')

Found 264 images belonging to 2 classes.
Found 112 images belonging to 2 classes.


## Block Stem

In [0]:

from keras.models import Model
from keras.layers import Activation, Input, concatenate, Dropout, Dense, Flatten
from keras.layers import MaxPooling2D, Conv2D, AveragePooling2D, BatchNormalization




def block_stem(inputs):
    net = conv2d(inputs, 32, (3, 3), strides=(2, 2), padding='valid')
    net = conv2d(net, 32, (3, 3), padding='valid')
    net = conv2d(net, 64, (3, 3))

    branch_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(net)
    branch_2 = conv2d(net, 96, (3, 3), strides=(2, 2), padding='valid')

    net = concatenate([branch_1, branch_2])

    branch_1 = conv2d(net, 64, (1, 1))
    branch_1 = conv2d(branch_1, 96, (3, 3), padding='valid')

    branch_2 = conv2d(net, 64, (1, 1))
    branch_2 = conv2d(branch_2, 64, (1, 7))
    branch_2 = conv2d(branch_2, 64, (7, 1))
    branch_2 = conv2d(branch_2, 96, (3, 3), padding='valid')

    net = concatenate([branch_1, branch_2])

    branch_1 = conv2d(net, 192, (3, 3), strides=(2, 2), padding='valid')  # different from the paper
    branch_2 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(net)

    net = concatenate([branch_1, branch_2])

    return net

## Block Inception a

In [0]:
def block_inception_a(inputs):
    branch_1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 96, (1, 1))


    branch_2 = conv2d(inputs, 96, (1, 1))

    branch_3 = conv2d(inputs, 64, (1, 1))
    branch_3 = conv2d(branch_3, 96, (3, 3))

    branch_4 = conv2d(inputs, 64, (1, 1))
    branch_4 = conv2d(branch_4, 96, (3, 3))
    branch_4 = conv2d(branch_4, 96, (3, 3))

    return concatenate([branch_1, branch_2, branch_3, branch_4])

## Block inception B

In [0]:

def block_inception_b(inputs):
    branch_1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 128, (1, 1))

    branch_2 = conv2d(inputs, 384, (1, 1))

    branch_3 = conv2d(inputs, 192, (1, 1))
    branch_3 = conv2d(branch_3, 224, (1, 7))
    branch_3 = conv2d(branch_3, 256, (7, 1))  # different from the paper

    branch_4 = conv2d(inputs, 192, (1, 1))
    branch_4 = conv2d(branch_4, 192, (1, 7))
    branch_4 = conv2d(branch_4, 224, (7, 1))
    branch_4 = conv2d(branch_4, 224, (1, 7))
    branch_4 = conv2d(branch_4, 256, (7, 1))

    return concatenate([branch_1, branch_2, branch_3, branch_4])

## Block Inception C

In [0]:

def block_inception_c(inputs):
    branch_1 = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 256, (1, 1))

    branch_2 = conv2d(inputs, 256, (1, 1))

    branch_3 = conv2d(inputs, 384, (1, 1))
    branch_3_1 = conv2d(branch_3, 256, (1, 3))
    branch_3_2 = conv2d(branch_3, 256, (3, 1))

    branch_4 = conv2d(inputs, 384, (1, 1))
    branch_4 = conv2d(branch_4, 512, (3, 1))
    branch_4 = conv2d(branch_4, 448, (1, 3))
    branch_4_2 = conv2d(branch_4, 256, (1, 3))
    branch_4_1 = conv2d(branch_4, 256, (3, 1))

    return concatenate([branch_1, branch_2, branch_3_1, branch_3_2, branch_4_1, branch_4_2])


## Block Reduction A

In [0]:
def block_reduction_a(inputs):
    branch_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(inputs)

    branch_2 = conv2d(inputs, 384, (3, 3), strides=(2, 2), padding='valid')

    branch_3 = conv2d(inputs, 192, (1, 1))
    branch_3 = conv2d(branch_3, 224, (3, 3))
    branch_3 = conv2d(branch_3, 256, (3, 3), strides=(2, 2), padding='valid')

    return concatenate([branch_1, branch_2, branch_3])

## Block Reduction B

In [0]:
def block_reduction_b(inputs):
    branch_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(inputs)

    branch_2 = conv2d(inputs, 192, (1, 1))
    branch_2 = conv2d(branch_2, 192, (3, 3), strides=(2, 2), padding='valid')

    branch_3 = conv2d(inputs, 256, (1, 1))
    branch_3 = conv2d(branch_3, 256, (1, 7))
    branch_3 = conv2d(branch_3, 320, (7, 1))
    branch_3 = conv2d(branch_3, 320, (3, 3), strides=(2, 2), padding='valid')

    return concatenate([branch_1, branch_2, branch_3])

In [0]:

def conv2d(net, filters, kernel_size, strides=(1, 1), padding='same'):
    net = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, use_bias=False)(net)
    net = BatchNormalization()(net)
    net = Activation('relu')(net)
    return net

def create(classes_num=1, image_height=299, image_width=299, image_channel=3):
    inputs = Input((image_height, image_width, image_channel))

    # 299 x 299 x 3
    net = block_stem(inputs)

    # 4 x Inception-A ( Output: 35 x 35 x 384 )
    for i in range(4):
        net = block_inception_a(net)

    # Reduction-A ( Output: 17 x 17 x 1024 )
    net = block_reduction_a(net)

    # 7 x Inception-B ( Output: 17 x 17 x 1024 )
    for i in range(7):
        net = block_inception_b(net)

    # Reduction-B ( Output: 8 x 8 x 1536 )
    net = block_reduction_b(net)

    # 3 x Inception-C ( Output: 8 x 8 x 1536 )
    for i in range(3):
        net = block_inception_c(net)

    # Average Pooling ( Output: 1536 )
    net = AveragePooling2D((8, 8))(net)

    # Dropout ( keep 0.8 )
    net = Dropout(0.2)(net)
    net = Flatten()(net)

    # Output
    outputs = Dense(units=1, activation='sigmoid')(net)

    return Model(inputs, outputs, name='Inception-v4')

# Create model

In [0]:
model = create()

# Model summary

In [0]:
model.summary()


Model: "Inception-v4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1044 (Conv2D)            (None, 149, 149, 32) 864         input_10[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1044 (Batch (None, 149, 149, 32) 128         conv2d_1044[0][0]                
__________________________________________________________________________________________________
activation_1044 (Activation)    (None, 149, 149, 32) 0           batch_normalization_1044[0][0]   
_______________________________________________________________________________________

In [0]:
model.compile(Adam(lr=0.0001), 
              loss='binary_crossentropy',
              metrics=['accuracy'])
#model.fit(train_generator,validation_data=validation_generator,batch_size=16,epochs=EPOCHS,verbose=1)

In [0]:
model.fit(train_generator,validation_data=validation_generator,epochs=EPOCHS)

Epoch 1/41
44/44 [==============================] - 77s 2s/step - loss: 0.6340 - accuracy: 0.7045 - val_loss: 1.2969 - val_accuracy: 0.5893
Epoch 2/41
44/44 [==============================] - 36s 823ms/step - loss: 0.3741 - accuracy: 0.8068 - val_loss: 2.8778 - val_accuracy: 0.5893
Epoch 3/41
44/44 [==============================] - 36s 826ms/step - loss: 0.3251 - accuracy: 0.8409 - val_loss: 3.3370 - val_accuracy: 0.5893
Epoch 4/41
44/44 [==============================] - 36s 825ms/step - loss: 0.3207 - accuracy: 0.8485 - val_loss: 4.8466 - val_accuracy: 0.5893
Epoch 5/41
44/44 [==============================] - 36s 825ms/step - loss: 0.1769 - accuracy: 0.9242 - val_loss: 4.2658 - val_accuracy: 0.5893
Epoch 6/41
44/44 [==============================] - 36s 815ms/step - loss: 0.1947 - accuracy: 0.9318 - val_loss: 3.4701 - val_accuracy: 0.5893
Epoch 7/41
44/44 [==============================] - 36s 821ms/step - loss: 0.1951 - accuracy: 0.9432 - val_loss: 3.0395 - val_accuracy: 0.5893
Ep